In [12]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

import os

In [8]:
# Initialize SparkSession
os.environ['SPARK_HOME'] = 'C:/spark-3.5.0-bin-hadoop3'
os.environ['PATH'] += 'C:/spark-3.5.0-bin-hadoop3/bin'
spark = SparkSession.builder \
    .appName("Wind Power Prediction") \
    .getOrCreate()


In [9]:
# Load data
df = spark.read.csv('../Dataset/T1.csv', header=True, inferSchema=True)


In [10]:
# Remove irrelevant columns
df = df.drop('Date/Time', 'Theoretical_Power_Curve (KWh)')

In [27]:
print(df)

DataFrame[LV ActivePower (kW): double, Wind Speed (m/s): double, Wind Direction (°): double]


In [35]:
# Normalize features
feature_columns = ['Wind Direction (°)', 'Wind Speed (m/s)']
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
scaler = MinMaxScaler(inputCol='features', outputCol='scaled_features')

In [36]:
# Define models
models = {
    'XGBoost': GBTRegressor(labelCol='LV ActivePower (kW)'),
    'Random Forest': RandomForestRegressor(labelCol='LV ActivePower (kW)'),
    'Linear Regression': LinearRegression(labelCol='LV ActivePower (kW)'),
    'Decision Tree': DecisionTreeRegressor(labelCol='LV ActivePower (kW)')
}

In [37]:
# Define evaluator
evaluator = RegressionEvaluator(labelCol="LV ActivePower (kW)", predictionCol="prediction", metricName="r2")

In [38]:
# Split data into train and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [39]:
for name, model in models.items():
    pipeline = Pipeline(stages=[assembler, scaler, model])
    paramGrid = ParamGridBuilder().build()
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=5)
    cvModel = crossval.fit(train_data)
    predictions = cvModel.transform(test_data)
    r2 = evaluator.evaluate(predictions)
    print(f'R2-{name}: {r2}')

R2-XGBoost: 0.9166769677417363
R2-Random Forest: 0.8933770685698812
R2-Linear Regression: 0.8335548083227226
R2-Decision Tree: 0.914335774900805


In [ ]:
# Stop SparkSession
spark.stop()